# Chapter 4, Orthogonality

## 4.1 Orthogonality of the Four Spaces

이 Chapter에서는 이전에 다루었던 Four Spaces들이 Orthogonal하다는 성질을 가진 다는 것에 대해 다룹니다.
Row space와 Nullspace를 통해 두 subspace의 orthogonality 확인 정도만 하겠습니다

In [1]:
import sys

def make_rref(matrix, pivot_cols):
    col_of_pivot = 0
    i = 0
    
    while i < len(matrix) and col_of_pivot < len(matrix[0]):
        pivot = matrix[i][col_of_pivot]
        
        if pivot == 0:#0인 경우 pivot이 아니니 다음 열로 넘어가 탐색한다.
            col_of_pivot += 1
        else:
            pivot_cols.append(col_of_pivot)# pivot이니 pivot_cols에 저장한다.
            for j in range(i + 1, len(matrix)):
                multiplier = matrix[j][col_of_pivot] / pivot
                for k in range(col_of_pivot, len(matrix[0])):
                    matrix[j][k] -= (matrix[i][k] * multiplier)
            i += 1  # 다음 행으로 이동
            col_of_pivot += 1
    #위에서 아래로 elimination
    
    
    index = 0
    for pivot_col in pivot_cols:
        if matrix[index][pivot_col] != 1:
            multiplier = matrix[index][pivot_col]
            for col in range(len(matrix[0])):
                matrix[index][col] /= multiplier
        index+=1
    #pivot 값 모두 1로 만들기
    
    pivot_cols.reverse()
    
    index = 0
    i = len(matrix) -1
    while i >= 0 and index<len(pivot_cols):
        pivot = matrix[i][pivot_cols[index]]
        
        if pivot != 1:#1인 경우 pivot이 아니니 다음 행로 넘어가 탐색한다.
            i -= 1
        else:
            for j in range(i - 1, -1, -1):
                multiplier = matrix[j][pivot_cols[index]] / pivot
                for k in range(0, len(matrix[0])):
                    matrix[j][k] -= (matrix[i][k] * multiplier)
            i -= 1  # 다음 열으로 이동
            index += 1
    
    #아래에서 위로 elimination

    
    return matrix
    #성공적으로 Elimination을 했다면 True를 반환해 Elimination된 Matrix를 출력한다
    
def print_matrix(matrix):
    rows = len(matrix)
    cols = len(matrix[0])
    
    for i in range(rows):
        for j in range(cols):
            print(matrix[i][j], end ="\t")
        print()
    
def make_nullspace(matrix, pivot_cols, free_cols):
    cols = len(matrix[0]) - len(pivot_cols)
    rows = len(matrix[0])
    #Nullspace는 원 matrix의 rows, cols수가 반대
    
    nullspace = list()
    
    for _ in range(rows):
        temp = [0]*cols
        nullspace.append(temp)
    
    free_index = 0
    pivot_index = 0
    index = 0
    
    for row in range(rows):
        if row in pivot_cols:
            index = 0
            for col in free_cols:
                nullspace[row][index]= -matrix[pivot_index][col]
                index+=1
            pivot_index += 1
            #Nullspace의 행이 not free하다면 RRef의 같은 행의 free columns들을 가져옴
        else:
            nullspace[row][free_index] = 1
            free_index+=1
            #아니라면 Identity matrix처럼 1로 만듦
            
    return nullspace

"""
    위 알고리즘은 R = | I F | 일 때, N = | -F | 를 가진다는 관계를 통해 구현되었습니다.
                      | 0 0 |            |  I |
"""


def Matrice_multiplication(Ma, Mb):
    if(len(Ma[0])!=len(Mb)):
        print("Invalid Multiplication")
    else:
        answer = list()
        
        for i in range(len(Ma)):
            temp = [0.0]*len(Mb[0])
            answer.append(temp)
            
        #answer을 위한 Matrix를 초기화한다.
        
        for i in range(len(Ma)):
            for j in range(len(Mb[0])):
                temp = 0
                for k in range(len(Mb)):
                    temp += Ma[i][k]*Mb[k][j]#A의 행, B열의 열을 곱하는 가장 기초적인 방법
                answer[i][j] = temp
                
        return answer
    
original = list()
matrix = list()

rows = int(input("Matrix A 행 수를 입력하세요: "))
cols = int(input("Matrix A 열 수를 입력하세요: "))

for i in range(rows):
        row = input(f"행 {i+1}을 공백으로 구분하여 입력하세요: ").split()
        if len(row) != cols:
            print("열 수와 입력된 값의 개수가 일치하지 않습니다.")
            sys.exit(1)
        else:
            matrix.append([float(val) for val in row])
            original.append([float(val) for val in row])
print()


pivot_cols = list() #pivot column들의 열번호를 저장한다.
make_rref(matrix, pivot_cols)
free_cols = [x for x in range(0, cols) if x not in pivot_cols]# free columns들의 열번호를 저장한다.

N_matrix = make_nullspace(matrix, pivot_cols, free_cols)

print("Nullspace : ")
print_matrix(N_matrix)
print()

print("Rowspace와 Nullspace의 곱 : ")
print(Matrice_multiplication(original, N_matrix))





Matrix A 행 수를 입력하세요: 2
Matrix A 열 수를 입력하세요: 3
행 1을 공백으로 구분하여 입력하세요: 1 3 4
행 2을 공백으로 구분하여 입력하세요: 5 2 7

Nullspace : 
-1.0	
-1.0	
1	

Rowspace와 Nullspace의 곱 : 
[[0.0], [0.0]]


프로그램 구현상에서도 서로 직교함을 알 수 있습니다.

## 4.2 Projections

이 chapter에서는 Projection에 대해 다룹니다. 

Projection matrix, Projection된 벡터, $\widehat{x}$을 구현해 보겠습니다.
이때까지 한 행렬 곱, 역행렬, Transpose 연산들을 이용해서 공식 그대로 구현할 수 있습니다.

In [7]:
def Matrice_multiplication(Ma, Mb):
    if(len(Ma[0])!=len(Mb)):
        print("Invalid Multiplication")
    else:
        answer = list()
        
        for i in range(len(Ma)):
            temp = [0.0]*len(Mb[0])
            answer.append(temp)
            
        #answer을 위한 Matrix를 초기화한다.
        
        for i in range(len(Ma)):
            for j in range(len(Mb[0])):
                temp = 0
                for k in range(len(Mb)):
                    temp += Ma[i][k]*Mb[k][j]#A의 행, B열의 열을 곱하는 가장 기초적인 방법
                answer[i][j] = temp
                
        return answer


def print_matrix(matrix):
    rows = len(matrix)
    cols = len(matrix[0])
    
    for i in range(rows):
        for j in range(cols):
            print(matrix[i][j], end ="\t")
        print()
        
def transpose(Ma):
    answer = list()
    for _ in range(len(Ma[0])):
        temp = [0.0]*len(Ma)
        answer.append(temp)
        
    for i in range(len(Ma)):
        for j in range(len(Ma[0])):
            answer[j][i] = Ma[i][j]
    return answer

def Gauss_Jordan(matrix, matrix2):
    for i in range(len(matrix)):
        pivot = matrix[i][i] ## pivot을 구한다.
        
        if pivot == 0:
            print(f"{i+1}행 {i+1}열의 pivot이 0입니다!")
            return None
            #pivot이 0이면 False를 반환해 Failure을 출력한다.
        else:
            for j in range(i+1, len(matrix)):
                multiplier = matrix[j][i]/pivot
                
                #각 행의 pivot에 대한 multiplier를 구한다
                
                for k in range(0, len(matrix[0])):
                    matrix[j][k] -= (matrix[i][k] * multiplier)
                    matrix2[j][k] -= (matrix2[i][k] * multiplier)
                    #multiplier를 이용해서 matrix, matrix2의 각 행을 Elimination해준다.
                    
    #여기서 끝나지 않고 아래에서 위로 한번더 빼준다.
    for i in range(len(matrix)-1, -1 , -1):
        pivot = matrix[i][i] ## pivot을 구한다.
        
        for j in range(i-1, -1, -1):
            multiplier = matrix[j][i]/pivot
                
            #각 행의 pivot에 대한 multiplier를 구한다
                
            for k in range(0, len(matrix[0])):
                matrix[j][k] -= (matrix[i][k] * multiplier)
                matrix2[j][k] -= (matrix2[i][k] * multiplier)
                #multiplier를 이용해서 matrix, matrix2의 각 행을 Elimination해준다.   
        
    #대각성분을 1로 만들어준다.
    for i in range(len(matrix)):
        pivot = matrix[i][i]
        matrix[i][i] /= pivot
            
        for j in range(len(matrix)):
            matrix2[i][j]/= pivot
        
    return matrix2
            #성공적으로 Elimination을 했다면 True를 반환해 Elimination된 Matrix를 출력한다

def make_identity(rows, cols): #Gauss-Jordan Elimination을 위해 Identity matrix를 만든다.
    Identity = list()
    for i in range(rows):
        row_Identity = list()
        
        for j in range(cols):
            if(i != j):
                row_Identity.append(0)
            else:
                row_Identity.append(1) # 대각행렬에는 1, 아니면 0을 입력하여 반환한다.
        
        Identity.append(row_Identity)

    return Identity


def find_xhat(Ma, Mb):
    A = Ma.copy()
    AT = transpose(Ma.copy())
    AT_A = Matrice_multiplication(AT, A)
    
    inv_AT_A = make_identity(len(AT_A), len(AT_A))
    
    Gauss_Jordan(AT_A, inv_AT_A)
    
    temp = Matrice_multiplication(inv_AT_A, AT)
    
    return Matrice_multiplication(temp, Mb)

def find_p(Ma, Mb):
    return Matrice_multiplication(Ma.copy(), find_xhat(Ma, Mb))

def find_P(Ma):
    A = Ma.copy()
    AT = transpose(Ma.copy())
    AT_A = Matrice_multiplication(AT, A)
    
    inv_AT_A = make_identity(len(AT_A), len(AT_A))
    
    Gauss_Jordan(AT_A, inv_AT_A)
    
    temp = Matrice_multiplication(inv_AT_A, AT)
    
    return Matrice_multiplication(A, temp)


matrix = list()

rows = int(input("Matrix A 행 수를 입력하세요: "))
cols = int(input("Matrix A 열 수를 입력하세요: "))

for i in range(rows):
        row = input(f"행 {i+1}을 공백으로 구분하여 입력하세요: ").split()
        if len(row) != cols:
            print("열 수와 입력된 값의 개수가 일치하지 않습니다.")
            sys.exit(1)
        else:
            matrix.append([float(val) for val in row])
print()

matrix2 = list()

rows = int(input("Matrix B 행 수를 입력하세요: "))
cols = int(input("Matrix B 열 수를 입력하세요: "))

for i in range(rows):
        row = input(f"행 {i+1}을 공백으로 구분하여 입력하세요: ").split()
        if len(row) != cols:
            print("열 수와 입력된 값의 개수가 일치하지 않습니다.")
            sys.exit(1)
        else:
            matrix2.append([float(val) for val in row])
print()

print("xhat : ")
print_matrix(find_xhat(matrix, matrix2))
print()
      
print("p : ")
print_matrix(find_p(matrix, matrix2))
print()
 
print("P : ")
print_matrix(find_P(matrix))
print()
 


Matrix A 행 수를 입력하세요: 3
Matrix A 열 수를 입력하세요: 2
행 1을 공백으로 구분하여 입력하세요: 1 0
행 2을 공백으로 구분하여 입력하세요: 1 1
행 3을 공백으로 구분하여 입력하세요: 1 2

Matrix B 행 수를 입력하세요: 3
Matrix B 열 수를 입력하세요: 1
행 1을 공백으로 구분하여 입력하세요: 6
행 2을 공백으로 구분하여 입력하세요: 0
행 3을 공백으로 구분하여 입력하세요: 0

xhat : 
5.0	
-3.0	

p : 
5.0	
2.0	
-1.0	

P : 
0.8333333333333334	0.33333333333333337	-0.16666666666666663	
0.33333333333333337	0.33333333333333337	0.33333333333333337	
-0.16666666666666663	0.33333333333333337	0.8333333333333334	



## 4.3 Least Square Approximation

이 장에서는 공학적으로 특정 포인트의 경향을 보이는 추세선을 만들고, 그 포인트 간의 오차를 출력하는 방법론에 대해 다룹니다.
책의 예제 하나만 다루어 보도록 하겠습니다.

1. 모델링

상황은 (0,6), (1,0), (2.0)에 대한 가장 가까운(오차가 적은) 직선을 만드는 것입니다.
우선 b = C + Dt 라는 직선을 가정하고 그에 따라

$$A = \begin{pmatrix} 1 & 0 \\ 1 & 1 \\ 1 & 2 \end{pmatrix}$$ 

$$x = \begin{pmatrix} C \\  D \end{pmatrix}$$

$$b = \begin{pmatrix} 6 \\ 0 \\ 0 \end{pmatrix}$$
    
으로 설계합니다.

In [21]:
import sys


def make_rref(matrix, pivot_cols):
    col_of_pivot = 0
    i = 0
    
    while i < len(matrix) and col_of_pivot < len(matrix[0])-1:
        pivot = matrix[i][col_of_pivot]
        
        if pivot == 0:#0인 경우 pivot이 아니니 다음 열로 넘어가 탐색한다.
            col_of_pivot += 1
        else:
            pivot_cols.append(col_of_pivot)# pivot이니 pivot_cols에 저장한다.
            for j in range(i + 1, len(matrix)):
                multiplier = matrix[j][col_of_pivot] / pivot
                for k in range(col_of_pivot, len(matrix[0])):
                    matrix[j][k] -= (matrix[i][k] * multiplier)
            i += 1  # 다음 행으로 이동
            col_of_pivot += 1
    #위에서 아래로 elimination
    
    
    index = 0
    for pivot_col in pivot_cols:
        if matrix[index][pivot_col] != 1:
            multiplier = matrix[index][pivot_col]
            for col in range(len(matrix[0])):
                matrix[index][col] /= multiplier
        index+=1
    #pivot 값 모두 1로 만들기
    
    pivot_cols.reverse()
    
    index = 0
    i = len(matrix) -1
    while i >= 0 and index<len(pivot_cols):
        pivot = matrix[i][pivot_cols[index]]
        
        if pivot != 1:#1인 경우 pivot이 아니니 다음 행로 넘어가 탐색한다.
            i -= 1
        else:
            for j in range(i - 1, -1, -1):
                multiplier = matrix[j][pivot_cols[index]] / pivot
                for k in range(0, len(matrix[0])):
                    matrix[j][k] -= (matrix[i][k] * multiplier)
            i -= 1  # 다음 열으로 이동
            index += 1
    
    #아래에서 위로 elimination

    pivot_cols.reverse()
    
    return matrix
    #성공적으로 Elimination을 했다면 True를 반환해 Elimination된 Matrix를 출력한다
    
def print_matrix(matrix):
    rows = len(matrix)
    cols = len(matrix[0])
    
    for i in range(rows):
        for j in range(cols):
            print(matrix[i][j], end ="\t")
        print()

def find_particular(matrix, pivot_cols):
    answer = [0]*(len(matrix[0])-1)
    
    target = list()
    
    for i in range(len(matrix)):
        target.append(matrix[i][len(matrix[0])-1])
    #augmented matrix의 마지막 부분(Rx = b에서 b만 저장)
    
    index = 0
    i = 0
    
    while True:
            
        flag = True
        for j in range(len(matrix[0])-1):
            if matrix[i][j] != 0:
                flag = False
                break
            #만약 rref의 특정 행의 모든 열이 0이면 True, 아니면 False
            
        if flag and target[i] != 0: #rref의 특정 행의 모든 열이 0인데, b부분의 값이 0이 아닌경우
            return None #해가 없으므로 None 반환
        
        else:
            if i == len(matrix) -1 and index == len(matrix[0])-2 :
                break
            
            if index in pivot_cols:
                answer[index] = target[i] 
                if index +1 < len(matrix[0]) -1 :
                    index+=1
                if i +1 < len(matrix) :
                    i+=1
            else:
                answer[index] = 0
                if index +1 < len(matrix[0]) -1 :
                    index+=1

    return answer
    
matrix = list()

rows = int(input("Augmented Matrix A 행 수를 입력하세요: "))
cols = int(input("Augmented Matrix A 열 수를 입력하세요: "))

for i in range(rows):
        row = input(f"행 {i+1}을 공백으로 구분하여 입력하세요: ").split()
        if len(row) != cols:
            print("열 수와 입력된 값의 개수가 일치하지 않습니다.")
            sys.exit(1)
        else:
            matrix.append([float(val) for val in row])
print()


pivot_cols = list() #pivot column들의 열번호를 저장한다.
make_rref(matrix, pivot_cols)
free_cols = [x for x in range(0, cols-1) if x not in pivot_cols]# free columns들의 열번호를 저장한다.

r = len(pivot_cols)
m = rows
n = cols -1
particular_solution = find_particular(matrix, pivot_cols)

if r == m and r == n:
    print("particular solution : ")
    print(particular_solution)
    print()
    print("There is only one Nullspace")

elif r == m and r < n:
    print("particular solution : ")
    print(particular_solution)
    print()
    print("There are infinite combination of Nullspace")

elif r < m and r== n:
    if particular_solution :
        print("particular solution : ")
        print(particular_solution)
        print()
        print("There is only one Nullspace")
    else:
        print("There is no solution")
    
elif r < m and r < n :
    if particular_solution :
        print("particular solution : ")
        print(particular_solution)
        print()
        print("There are infinite combination of Nullspace")
    else:
        print("There is no solution")

Augmented Matrix A 행 수를 입력하세요: 3
Augmented Matrix A 열 수를 입력하세요: 3
행 1을 공백으로 구분하여 입력하세요: 1 0 6
행 2을 공백으로 구분하여 입력하세요: 1 1 0
행 3을 공백으로 구분하여 입력하세요: 1 2 0

There is no solution


하지만 이에 해당하는 해는 구할 수 없고, 근사가 필요합니다.
이에 Projection을 써서 x대신 $\widehat{x}$을 구합니다.

In [22]:
def Matrice_multiplication(Ma, Mb):
    if(len(Ma[0])!=len(Mb)):
        print("Invalid Multiplication")
    else:
        answer = list()
        
        for i in range(len(Ma)):
            temp = [0.0]*len(Mb[0])
            answer.append(temp)
            
        #answer을 위한 Matrix를 초기화한다.
        
        for i in range(len(Ma)):
            for j in range(len(Mb[0])):
                temp = 0
                for k in range(len(Mb)):
                    temp += Ma[i][k]*Mb[k][j]#A의 행, B열의 열을 곱하는 가장 기초적인 방법
                answer[i][j] = temp
                
        return answer


def print_matrix(matrix):
    rows = len(matrix)
    cols = len(matrix[0])
    
    for i in range(rows):
        for j in range(cols):
            print(matrix[i][j], end ="\t")
        print()
        
def transpose(Ma):
    answer = list()
    for _ in range(len(Ma[0])):
        temp = [0.0]*len(Ma)
        answer.append(temp)
        
    for i in range(len(Ma)):
        for j in range(len(Ma[0])):
            answer[j][i] = Ma[i][j]
    return answer

def Gauss_Jordan(matrix, matrix2):
    for i in range(len(matrix)):
        pivot = matrix[i][i] ## pivot을 구한다.
        
        if pivot == 0:
            print(f"{i+1}행 {i+1}열의 pivot이 0입니다!")
            return None
            #pivot이 0이면 False를 반환해 Failure을 출력한다.
        else:
            for j in range(i+1, len(matrix)):
                multiplier = matrix[j][i]/pivot
                
                #각 행의 pivot에 대한 multiplier를 구한다
                
                for k in range(0, len(matrix[0])):
                    matrix[j][k] -= (matrix[i][k] * multiplier)
                    matrix2[j][k] -= (matrix2[i][k] * multiplier)
                    #multiplier를 이용해서 matrix, matrix2의 각 행을 Elimination해준다.
                    
    #여기서 끝나지 않고 아래에서 위로 한번더 빼준다.
    for i in range(len(matrix)-1, -1 , -1):
        pivot = matrix[i][i] ## pivot을 구한다.
        
        for j in range(i-1, -1, -1):
            multiplier = matrix[j][i]/pivot
                
            #각 행의 pivot에 대한 multiplier를 구한다
                
            for k in range(0, len(matrix[0])):
                matrix[j][k] -= (matrix[i][k] * multiplier)
                matrix2[j][k] -= (matrix2[i][k] * multiplier)
                #multiplier를 이용해서 matrix, matrix2의 각 행을 Elimination해준다.   
        
    #대각성분을 1로 만들어준다.
    for i in range(len(matrix)):
        pivot = matrix[i][i]
        matrix[i][i] /= pivot
            
        for j in range(len(matrix)):
            matrix2[i][j]/= pivot
        
    return matrix2
            #성공적으로 Elimination을 했다면 True를 반환해 Elimination된 Matrix를 출력한다

def make_identity(rows, cols): #Gauss-Jordan Elimination을 위해 Identity matrix를 만든다.
    Identity = list()
    for i in range(rows):
        row_Identity = list()
        
        for j in range(cols):
            if(i != j):
                row_Identity.append(0)
            else:
                row_Identity.append(1) # 대각행렬에는 1, 아니면 0을 입력하여 반환한다.
        
        Identity.append(row_Identity)

    return Identity


def find_xhat(Ma, Mb):
    A = Ma.copy()
    AT = transpose(Ma.copy())
    AT_A = Matrice_multiplication(AT, A)
    
    inv_AT_A = make_identity(len(AT_A), len(AT_A))
    
    Gauss_Jordan(AT_A, inv_AT_A)
    
    temp = Matrice_multiplication(inv_AT_A, AT)
    
    return Matrice_multiplication(temp, Mb)

def find_p(Ma, Mb):
    return Matrice_multiplication(Ma.copy(), find_xhat(Ma, Mb))

def find_P(Ma):
    A = Ma.copy()
    AT = transpose(Ma.copy())
    AT_A = Matrice_multiplication(AT, A)
    
    inv_AT_A = make_identity(len(AT_A), len(AT_A))
    
    Gauss_Jordan(AT_A, inv_AT_A)
    
    temp = Matrice_multiplication(inv_AT_A, AT)
    
    return Matrice_multiplication(A, temp)


matrix = list()

rows = int(input("Matrix A 행 수를 입력하세요: "))
cols = int(input("Matrix A 열 수를 입력하세요: "))

for i in range(rows):
        row = input(f"행 {i+1}을 공백으로 구분하여 입력하세요: ").split()
        if len(row) != cols:
            print("열 수와 입력된 값의 개수가 일치하지 않습니다.")
            sys.exit(1)
        else:
            matrix.append([float(val) for val in row])
print()

matrix2 = list()

rows = int(input("Matrix B 행 수를 입력하세요: "))
cols = int(input("Matrix B 열 수를 입력하세요: "))

for i in range(rows):
        row = input(f"행 {i+1}을 공백으로 구분하여 입력하세요: ").split()
        if len(row) != cols:
            print("열 수와 입력된 값의 개수가 일치하지 않습니다.")
            sys.exit(1)
        else:
            matrix2.append([float(val) for val in row])
print()

print("xhat : ")
print_matrix(find_xhat(matrix, matrix2))
print()

Matrix A 행 수를 입력하세요: 3
Matrix A 열 수를 입력하세요: 2
행 1을 공백으로 구분하여 입력하세요: 1 0
행 2을 공백으로 구분하여 입력하세요: 1 1
행 3을 공백으로 구분하여 입력하세요: 1 2

Matrix B 행 수를 입력하세요: 3
Matrix B 열 수를 입력하세요: 1
행 1을 공백으로 구분하여 입력하세요: 6
행 2을 공백으로 구분하여 입력하세요: 0
행 3을 공백으로 구분하여 입력하세요: 0

xhat : 
5.0	
-3.0	



여기서 구한 xhat이 가장 오차가 작은 C, D를 가지게 됩니다.

오차(Least Square)를 구하기 위해서는 b - p한 e를 구하고, e의 length^2를 구하면 됩니다.

In [31]:
import math

def Matrice_multiplication(Ma, Mb):
    if(len(Ma[0])!=len(Mb)):
        print("Invalid Multiplication")
    else:
        answer = list()
        
        for i in range(len(Ma)):
            temp = [0.0]*len(Mb[0])
            answer.append(temp)
            
        #answer을 위한 Matrix를 초기화한다.
        
        for i in range(len(Ma)):
            for j in range(len(Mb[0])):
                temp = 0
                for k in range(len(Mb)):
                    temp += Ma[i][k]*Mb[k][j]#A의 행, B열의 열을 곱하는 가장 기초적인 방법
                answer[i][j] = temp
                
        return answer


def print_matrix(matrix):
    rows = len(matrix)
    cols = len(matrix[0])
    
    for i in range(rows):
        for j in range(cols):
            print(matrix[i][j], end ="\t")
        print()
        
def transpose(Ma):
    answer = list()
    for _ in range(len(Ma[0])):
        temp = [0.0]*len(Ma)
        answer.append(temp)
        
    for i in range(len(Ma)):
        for j in range(len(Ma[0])):
            answer[j][i] = Ma[i][j]
    return answer

def Gauss_Jordan(matrix, matrix2):
    for i in range(len(matrix)):
        pivot = matrix[i][i] ## pivot을 구한다.
        
        if pivot == 0:
            print(f"{i+1}행 {i+1}열의 pivot이 0입니다!")
            return None
            #pivot이 0이면 False를 반환해 Failure을 출력한다.
        else:
            for j in range(i+1, len(matrix)):
                multiplier = matrix[j][i]/pivot
                
                #각 행의 pivot에 대한 multiplier를 구한다
                
                for k in range(0, len(matrix[0])):
                    matrix[j][k] -= (matrix[i][k] * multiplier)
                    matrix2[j][k] -= (matrix2[i][k] * multiplier)
                    #multiplier를 이용해서 matrix, matrix2의 각 행을 Elimination해준다.
                    
    #여기서 끝나지 않고 아래에서 위로 한번더 빼준다.
    for i in range(len(matrix)-1, -1 , -1):
        pivot = matrix[i][i] ## pivot을 구한다.
        
        for j in range(i-1, -1, -1):
            multiplier = matrix[j][i]/pivot
                
            #각 행의 pivot에 대한 multiplier를 구한다
                
            for k in range(0, len(matrix[0])):
                matrix[j][k] -= (matrix[i][k] * multiplier)
                matrix2[j][k] -= (matrix2[i][k] * multiplier)
                #multiplier를 이용해서 matrix, matrix2의 각 행을 Elimination해준다.   
        
    #대각성분을 1로 만들어준다.
    for i in range(len(matrix)):
        pivot = matrix[i][i]
        matrix[i][i] /= pivot
            
        for j in range(len(matrix)):
            matrix2[i][j]/= pivot
        
    return matrix2
            #성공적으로 Elimination을 했다면 True를 반환해 Elimination된 Matrix를 출력한다

def make_identity(rows, cols): #Gauss-Jordan Elimination을 위해 Identity matrix를 만든다.
    Identity = list()
    for i in range(rows):
        row_Identity = list()
        
        for j in range(cols):
            if(i != j):
                row_Identity.append(0)
            else:
                row_Identity.append(1) # 대각행렬에는 1, 아니면 0을 입력하여 반환한다.
        
        Identity.append(row_Identity)

    return Identity


def find_xhat(Ma, Mb):
    A = Ma.copy()
    AT = transpose(Ma.copy())
    AT_A = Matrice_multiplication(AT, A)
    
    inv_AT_A = make_identity(len(AT_A), len(AT_A))
    
    Gauss_Jordan(AT_A, inv_AT_A)
    
    temp = Matrice_multiplication(inv_AT_A, AT)
    
    return Matrice_multiplication(temp, Mb)

def find_p(Ma, Mb):
    return Matrice_multiplication(Ma.copy(), find_xhat(Ma, Mb))

def vector_addition(Va, Vb):
    if len(Va) != len(Vb):
        print("Invalid vector addition!")
        # 두 벡터의 components 개수가 같아야 계산이 가능함.
    else:
        answer = list()
        for i in range(len(Va)):
            answer.append(Va[i] + Vb[i])
        return answer; # 벡터끼리 값을 더해서 반환
    
def vector_multiplication(Va, c):
        answer = list()
        for i in range(len(Va)):
            answer.append(Va[i] * c)
        return answer; # 벡터에 scalar를 곱해서 반환

def dot_product(Va, Vb):
    answer = 0
    
    if len(Va) != len(Vb):
        print("Invalid dot product!")
        # 두 벡터의 components 개수가 같아야 계산이 가능함.
    else:
        answer = 0
        for i in range(len(Va)):
            answer += Va[i][0] * Vb[i][0]
    return answer #벡터의 덧셈 과정과 거의 같습니다. 다만 각각의 덧셈이 아닌 곱셈, 벡터가 아닌 스칼라를 반환한다는 점이 다릅니다.

def vector_length(Va):
    return math.sqrt(dot_product(Va, Va))
#정의대로 자기 자신을 내적후 제곱근을 취합니다

matrix = list()

rows = int(input("Matrix A 행 수를 입력하세요: "))
cols = int(input("Matrix A 열 수를 입력하세요: "))

for i in range(rows):
        row = input(f"행 {i+1}을 공백으로 구분하여 입력하세요: ").split()
        if len(row) != cols:
            print("열 수와 입력된 값의 개수가 일치하지 않습니다.")
            sys.exit(1)
        else:
            matrix.append([float(val) for val in row])
print()

matrix2 = list()

rows = int(input("Matrix B 행 수를 입력하세요: "))
cols = int(input("Matrix B 열 수를 입력하세요: "))

for i in range(rows):
        row = input(f"행 {i+1}을 공백으로 구분하여 입력하세요: ").split()
        if len(row) != cols:
            print("열 수와 입력된 값의 개수가 일치하지 않습니다.")
            sys.exit(1)
        else:
            matrix2.append([float(val) for val in row])
print()

print("p : ")
p = find_p(matrix, matrix2)
print_matrix(p)
print()

print("e :")
e = vector_addition(matrix2, vector_multiplication(p, -1))
print_matrix(e)
print()

print("Least squre of e : ")
print(dot_product(e, e))
print()


Matrix A 행 수를 입력하세요: 3
Matrix A 열 수를 입력하세요: 2
행 1을 공백으로 구분하여 입력하세요: 1 0
행 2을 공백으로 구분하여 입력하세요: 1 1
행 3을 공백으로 구분하여 입력하세요: 1 2

Matrix B 행 수를 입력하세요: 3
Matrix B 열 수를 입력하세요: 1
행 1을 공백으로 구분하여 입력하세요: 6
행 2을 공백으로 구분하여 입력하세요: 0
행 3을 공백으로 구분하여 입력하세요: 0

p : 
5.0	
2.0	
-1.0	

e :
6.0	
0.0	
0.0	

Least squre of e : 
36.0



## 4.4 Orthgonal Bases and Gram-Schmidt

이번 Chapter에서는 서로 다른 independent한 vector를 Orthonormal한 vector로 바꿔주는 Gram-Schmidt, QR 분해에 대해 다룹니다..

이 역시 위의 코드들을 이용해 공식을 구현하기만 하면 됩니다.

그 후 구한 orthonomral한 벡터들로 QR 분해를 하는 것을 구현하겠습니다.

교재에 나온 것은 3 by 3까지만 다루었지만, 일반화시켜 더 많은 n에 대해서도 다룰 수 있게 구현해보겠습니다.

In [67]:
def vector_addition(Va, Vb):
    if len(Va) != len(Vb):
        print("Invalid vector addition!")
        # 두 벡터의 components 개수가 같아야 계산이 가능함.
    else:
        answer = list()
        for i in range(len(Va)):
            answer.append(Va[i] + Vb[i])
        return answer; # 벡터끼리 값을 더해서 반환
    
def vector_multiplication(Va, c):
        answer = list()
        for i in range(len(Va)):
            answer.append(Va[i] * c)
        return answer; # 벡터에 scalar를 곱해서 반환

def Matrice_multiplication(Ma, Mb):
    if(len(Ma[0])!=len(Mb)):
        print("Invalid Multiplication")
    else:
        answer = list()
        
        for i in range(len(Ma)):
            temp = [0.0]*len(Mb[0])
            answer.append(temp)
            
        #answer을 위한 Matrix를 초기화한다.
        
        for i in range(len(Ma)):
            for j in range(len(Mb[0])):
                temp = 0
                for k in range(len(Mb)):
                    temp += Ma[i][k]*Mb[k][j]#A의 행, B열의 열을 곱하는 가장 기초적인 방법
                answer[i][j] = temp
                
        return answer


def print_matrix(matrix):
    rows = len(matrix)
    cols = len(matrix[0])
    
    for i in range(rows):
        for j in range(cols):
            print(matrix[i][j], end ="\t")
        print()
        
def transpose(Ma):
    answer = list()
    for _ in range(len(Ma[0])):
        temp = [0.0]*len(Ma)
        answer.append(temp)
        
    for i in range(len(Ma)):
        for j in range(len(Ma[0])):
            answer[j][i] = Ma[i][j]
    return answer

def Gauss_Jordan(matrix, matrix2):
    for i in range(len(matrix)):
        pivot = matrix[i][i] ## pivot을 구한다.
        
        if pivot == 0:
            print(f"{i+1}행 {i+1}열의 pivot이 0입니다!")
            return None
            #pivot이 0이면 False를 반환해 Failure을 출력한다.
        else:
            for j in range(i+1, len(matrix)):
                multiplier = matrix[j][i]/pivot
                
                #각 행의 pivot에 대한 multiplier를 구한다
                
                for k in range(0, len(matrix[0])):
                    matrix[j][k] -= (matrix[i][k] * multiplier)
                    matrix2[j][k] -= (matrix2[i][k] * multiplier)
                    #multiplier를 이용해서 matrix, matrix2의 각 행을 Elimination해준다.
                    
    #여기서 끝나지 않고 아래에서 위로 한번더 빼준다.
    for i in range(len(matrix)-1, -1 , -1):
        pivot = matrix[i][i] ## pivot을 구한다.
        
        for j in range(i-1, -1, -1):
            multiplier = matrix[j][i]/pivot
                
            #각 행의 pivot에 대한 multiplier를 구한다
                
            for k in range(0, len(matrix[0])):
                matrix[j][k] -= (matrix[i][k] * multiplier)
                matrix2[j][k] -= (matrix2[i][k] * multiplier)
                #multiplier를 이용해서 matrix, matrix2의 각 행을 Elimination해준다.   
        
    #대각성분을 1로 만들어준다.
    for i in range(len(matrix)):
        pivot = matrix[i][i]
        matrix[i][i] /= pivot
            
        for j in range(len(matrix)):
            matrix2[i][j]/= pivot
        
    return matrix2
            #성공적으로 Elimination을 했다면 True를 반환해 Elimination된 Matrix를 출력한다

def make_identity(rows, cols): #Gauss-Jordan Elimination을 위해 Identity matrix를 만든다.
    Identity = list()
    for i in range(rows):
        row_Identity = list()
        
        for j in range(cols):
            if(i != j):
                row_Identity.append(0)
            else:
                row_Identity.append(1) # 대각행렬에는 1, 아니면 0을 입력하여 반환한다.
        
        Identity.append(row_Identity)

    return Identity


def find_xhat(Ma, Mb):
    A = Ma.copy()
    AT = transpose(Ma.copy())
    AT_A = Matrice_multiplication(AT, A)
    
    inv_AT_A = make_identity(len(AT_A), len(AT_A))
    
    Gauss_Jordan(AT_A, inv_AT_A)
    
    temp = Matrice_multiplication(inv_AT_A, AT)
    
    return Matrice_multiplication(temp, Mb)

def dot_product(Va, Vb):
    answer = 0
    
    if len(Va) != len(Vb):
        print("Invalid dot product!")
        # 두 벡터의 components 개수가 같아야 계산이 가능함.
    else:
        answer = 0
        for i in range(len(Va)):
            answer += Va[i] * Vb[i]
    return answer #벡터의 덧셈 과정과 거의 같습니다. 다만 각각의 덧셈이 아닌 곱셈, 벡터가 아닌 스칼라를 반환한다는 점이 다릅니다.

def vector_length(Va):
    return math.sqrt(dot_product(Va, Va))
#정의대로 자기 자신을 내적후 제곱근을 취합니다


def unit_vector(Va):
    len_Va= vector_length(Va)
    answer = list()
    
    for i in range(len(Va)):
        answer.append(Va[i]/len_Va)
    
    return answer

def Gram_Schmidt(matrix):
    for i in range(len(matrix)):
        # 내부 리스트를 바로 벡터로 사용
        vector = matrix[i]
        
        for j in range(i):
            dot_product_result = dot_product(vector, matrix[j])
            projection = vector_multiplication(matrix[j], dot_product_result)
            vector = vector_addition(vector, vector_multiplication(projection, -1))

        matrix[i] = unit_vector(vector)

matrix = list()

rows = int(input("Gram-Schmidt 과정을 위한 벡터의 수를 입력하세요: "))
cols = int(input("벡터의 components 수를 입력하세요: "))

for i in range(rows):
        row = input(f"벡터 {i+1}을 공백으로 구분하여 입력하세요: ").split()
        if len(row) != cols:
            print("components 수와 입력된 값의 개수가 일치하지 않습니다.")
            sys.exit(1)
        else:
            matrix.append([float(val) for val in row]) 

Gram_Schmidt(matrix)
print()

for i in range(len(matrix)):
    print(f"벡터 {i+1} : {matrix[i]}")

Gram-Schmidt 과정을 위한 벡터의 수를 입력하세요: 3
벡터의 components 수를 입력하세요: 3
벡터 1을 공백으로 구분하여 입력하세요: 1 -1 0
벡터 2을 공백으로 구분하여 입력하세요: 2 0 -2
벡터 3을 공백으로 구분하여 입력하세요: 3 -3 3
벡터 1 : [0.7071067811865475, -0.7071067811865475, 0.0]
벡터 2 : [0.40824829046386313, 0.40824829046386296, -0.8164965809277261]
벡터 3 : [0.5773502691896263, 0.5773502691896255, 0.5773502691896255]


이를 바탕으로 QR 분해도 다루어 보겠습니다

In [73]:
def vector_addition(Va, Vb):
    if len(Va) != len(Vb):
        print("Invalid vector addition!")
        # 두 벡터의 components 개수가 같아야 계산이 가능함.
    else:
        answer = list()
        for i in range(len(Va)):
            answer.append(Va[i] + Vb[i])
        return answer; # 벡터끼리 값을 더해서 반환
    
def vector_multiplication(Va, c):
        answer = list()
        for i in range(len(Va)):
            answer.append(Va[i] * c)
        return answer; # 벡터에 scalar를 곱해서 반환

def Matrice_multiplication(Ma, Mb):
    if(len(Ma[0])!=len(Mb)):
        print("Invalid Multiplication")
    else:
        answer = list()
        
        for i in range(len(Ma)):
            temp = [0.0]*len(Mb[0])
            answer.append(temp)
            
        #answer을 위한 Matrix를 초기화한다.
        
        for i in range(len(Ma)):
            for j in range(len(Mb[0])):
                temp = 0
                for k in range(len(Mb)):
                    temp += Ma[i][k]*Mb[k][j]#A의 행, B열의 열을 곱하는 가장 기초적인 방법
                answer[i][j] = temp
                
        return answer


def print_matrix(matrix):
    rows = len(matrix)
    cols = len(matrix[0])
    
    for i in range(rows):
        for j in range(cols):
            print(matrix[i][j], end ="\t")
        print()
        
def transpose(Ma):
    answer = list()
    for _ in range(len(Ma[0])):
        temp = [0.0]*len(Ma)
        answer.append(temp)
        
    for i in range(len(Ma)):
        for j in range(len(Ma[0])):
            answer[j][i] = Ma[i][j]
    return answer

def Gauss_Jordan(matrix, matrix2):
    for i in range(len(matrix)):
        pivot = matrix[i][i] ## pivot을 구한다.
        
        if pivot == 0:
            print(f"{i+1}행 {i+1}열의 pivot이 0입니다!")
            return None
            #pivot이 0이면 False를 반환해 Failure을 출력한다.
        else:
            for j in range(i+1, len(matrix)):
                multiplier = matrix[j][i]/pivot
                
                #각 행의 pivot에 대한 multiplier를 구한다
                
                for k in range(0, len(matrix[0])):
                    matrix[j][k] -= (matrix[i][k] * multiplier)
                    matrix2[j][k] -= (matrix2[i][k] * multiplier)
                    #multiplier를 이용해서 matrix, matrix2의 각 행을 Elimination해준다.
                    
    #여기서 끝나지 않고 아래에서 위로 한번더 빼준다.
    for i in range(len(matrix)-1, -1 , -1):
        pivot = matrix[i][i] ## pivot을 구한다.
        
        for j in range(i-1, -1, -1):
            multiplier = matrix[j][i]/pivot
                
            #각 행의 pivot에 대한 multiplier를 구한다
                
            for k in range(0, len(matrix[0])):
                matrix[j][k] -= (matrix[i][k] * multiplier)
                matrix2[j][k] -= (matrix2[i][k] * multiplier)
                #multiplier를 이용해서 matrix, matrix2의 각 행을 Elimination해준다.   
        
    #대각성분을 1로 만들어준다.
    for i in range(len(matrix)):
        pivot = matrix[i][i]
        matrix[i][i] /= pivot
            
        for j in range(len(matrix)):
            matrix2[i][j]/= pivot
        
    return matrix2
            #성공적으로 Elimination을 했다면 True를 반환해 Elimination된 Matrix를 출력한다

def make_identity(rows, cols): #Gauss-Jordan Elimination을 위해 Identity matrix를 만든다.
    Identity = list()
    for i in range(rows):
        row_Identity = list()
        
        for j in range(cols):
            if(i != j):
                row_Identity.append(0)
            else:
                row_Identity.append(1) # 대각행렬에는 1, 아니면 0을 입력하여 반환한다.
        
        Identity.append(row_Identity)

    return Identity


def find_xhat(Ma, Mb):
    A = Ma.copy()
    AT = transpose(Ma.copy())
    AT_A = Matrice_multiplication(AT, A)
    
    inv_AT_A = make_identity(len(AT_A), len(AT_A))
    
    Gauss_Jordan(AT_A, inv_AT_A)
    
    temp = Matrice_multiplication(inv_AT_A, AT)
    
    return Matrice_multiplication(temp, Mb)

def dot_product(Va, Vb):
    answer = 0
    
    if len(Va) != len(Vb):
        print("Invalid dot product!")
        # 두 벡터의 components 개수가 같아야 계산이 가능함.
    else:
        answer = 0
        for i in range(len(Va)):
            answer += Va[i] * Vb[i]
    return answer #벡터의 덧셈 과정과 거의 같습니다. 다만 각각의 덧셈이 아닌 곱셈, 벡터가 아닌 스칼라를 반환한다는 점이 다릅니다.

def vector_length(Va):
    return math.sqrt(dot_product(Va, Va))
#정의대로 자기 자신을 내적후 제곱근을 취합니다


def unit_vector(Va):
    len_Va= vector_length(Va)
    answer = list()
    
    for i in range(len(Va)):
        answer.append(Va[i]/len_Va)
    
    return answer

def Gram_Schmidt(matrix):
    for i in range(len(matrix)):
        # 내부 리스트를 바로 벡터로 사용
        vector = matrix[i]
        
        for j in range(i):
            dot_product_result = dot_product(vector, matrix[j])
            projection = vector_multiplication(matrix[j], dot_product_result)
            vector = vector_addition(vector, vector_multiplication(projection, -1))

        matrix[i] = unit_vector(vector)

matrix = list()

rows = int(input("Matrix A 행 수를 입력하세요: "))
cols = int(input("Matrix A 열 수를 입력하세요: "))

for i in range(rows):
        row = input(f"행 {i+1}을 공백으로 구분하여 입력하세요: ").split()
        if len(row) != cols:
            print("열 수와 입력된 값의 개수가 일치하지 않습니다.")
            sys.exit(1)
        else:
            matrix.append([float(val) for val in row])
print()

original = matrix.copy()
AT = transpose(matrix)

Gram_Schmidt(AT)
Q = transpose(AT)
print()

print("A : ")
print_matrix(original)
print()

print("Q : ")
print_matrix(Q)
print()

print("R : ")
print_matrix(Matrice_multiplication(transpose(Q), original))
print()


Matrix A 행 수를 입력하세요: 3
Matrix A 열 수를 입력하세요: 3
행 1을 공백으로 구분하여 입력하세요: 1 2 3
행 2을 공백으로 구분하여 입력하세요: -1 0 -3
행 3을 공백으로 구분하여 입력하세요: 0 -2 3


A : 
1.0	2.0	3.0	
-1.0	0.0	-3.0	
0.0	-2.0	3.0	

Q : 
0.7071067811865475	0.40824829046386313	0.5773502691896263	
-0.7071067811865475	0.40824829046386296	0.5773502691896255	
0.0	-0.8164965809277261	0.5773502691896255	

R : 
1.414213562373095	1.414213562373095	4.242640687119285	
1.6653345369377348e-16	2.4494897427831788	-2.449489742783178	
7.771561172376096e-16	1.5543122344752192e-15	1.732050807568879	



원래 R 행렬은 upper triangle입니다. 그런데 R행렬을 보면 0이 아니라 e^-15 정도의 오차가 있는것을 확인할 수 있습니다.

이는 부동소수점에 의한 문제로, 실제값은 0.0000000000000001... 정도의 오차입니다.

이 부분에서 만약 절대값 0.00000001 이하의 값을 0으로 표기하게 바꾼다면 원하는 값을 구할 수 있습니다.